In [1]:
!pip install keras==2.2.4

In [2]:
!pip install tensorflow==2.5.0

     |████████████████████████████████| 454.4 MB 29 kB/s s eta 0:00:01     |██████████████▏                 | 201.5 MB 73.1 MB/s eta 0:00:04��█████████████▎              | 244.7 MB 74.8 MB/s eta 0:00:03     |██████████████████████          | 311.8 MB 80.5 MB/s eta 0:00:02        | 315.8 MB 80.5 MB/s eta 0:00:02��██▌     | 376.5 MB 51.3 MB/s eta 0:00:02     |██████████████████████████████  | 426.9 MB 74.4 MB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 78.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 64.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 70.4 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 72.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 31.1 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-11-17 15:14:40.321888: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow


In [4]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [5]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='wqDBzE9bN4SnEL6XUswnZeQS_55exh8NK80putx-L51r',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'na-donotdelete-pr-rwpgsl31avjqx7'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [7]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
%%bash
ls Dataset

photo-1589820296156-2454bb8a6ad1.jpg
TEST_SET
TRAIN_SET


In [9]:
pwd

'/home/wsuser/work'

In [10]:
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/TRAIN_SET',target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')

Found 4138 images belonging to 5 classes.


In [11]:
x_test=test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/TEST_SET',target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')

Found 1679 images belonging to 5 classes.


In [12]:
print(x_train.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [13]:
print(x_test.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [14]:
from collections import Counter as c
c(x_train.labels)

Counter({0: 995, 1: 1374, 2: 1019, 3: 275, 4: 475})

In [15]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
model=Sequential()

2022-11-17 15:14:47.430242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-11-17 15:14:47.430319: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)


In [17]:
model.add(Conv2D(32, (3, 3), input_shape=(64, 64 ,3), activation='relu'))


In [18]:
model.add(MaxPooling2D(pool_size=(2, 2)))


In [19]:
model.add(Conv2D(32, (3, 3), activation='relu'))

In [20]:
model.add(Flatten())


In [21]:
model.add(Dense(units=128, activation='relu'))


In [22]:
model.add(Dense(units=5, activation='softmax'))


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 26912)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3444864   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 3,455,653
Trainable params: 3,455,653
Non-trainable params: 0
______________________________________________

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [25]:
model.fit_generator(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=10)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
828/828 [==============================] - 84s 102ms/step - loss: 0.6347 - accuracy: 0.7595 - val_loss: 0.3015 - val_accuracy: 0.8827
Epoch 2/10
828/828 [==============================] - 81s 98ms/step - loss: 0.4375 - accuracy: 0.8369 - val_loss: 0.2802 - val_accuracy: 0.9005
Epoch 3/10
828/828 [==============================] - 80s 97ms/step - loss: 0.4089 - accuracy: 0.8490 - val_loss: 0.2844 - val_accuracy: 0.8892
Epoch 4/10
828/828 [==============================] - 80s 97ms/step - loss: 0.3668 - accuracy: 0.8630 - val_loss: 0.3460 - val_accuracy: 0.8862
Epoch 5/10
828/828 [==============================] - 81s 97ms/step - loss: 0.3609 - accuracy: 0.8618 - val_loss: 0.2221 - val_accuracy: 0.9178
Epoch 6/10
828/828 [==============================] - 82s 100ms/step - loss: 0.3434 - accuracy: 0.8666 - val_loss: 0.2624 - val_accuracy: 0.9035
Epoch 7/10
828/828 [==============================] - 82s 99ms/step - loss: 0.3154 - accuracy: 0.8809 - val_loss: 0.2497 - val_accurac

In [26]:
model.save('nutrition.h5')
#E1Gd7BpjkhiyKWKh3c42ETkeGG_8yCuKMsKbJ0WSaUm2

In [27]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image


In [28]:
model = load_model("nutrition.h5")

In [29]:
!tar -zcvf nutri-analyzer-model.tgz nutrition.h5

nutrition.h5


In [30]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 18.7 MB/s eta 0:00:01


In [31]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "E1Gd7BpjkhiyKWKh3c42ETkeGG_8yCuKMsKbJ0WSaUm2"
}

client = APIClient(wml_credentials)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/torch/package/_directory_reader.py:17: UserWarning: Failed to initialize NumPy: module compiled against API version 0xe but this version of numpy is 0xd (Triggered internally at  /opt/conda/conda-bld/pytorch-base_1658814980981/work/torch/csrc/utils/tensor_numpy.cpp:68.)
  _dtype_to_storage = {data_type(0).dtype: data_type for data_type in _storages}


In [32]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [33]:
space_uid = guid_from_space_name(client, 'NA')
print("Space UID : ", space_uid)

Space UID :  8b8f63d5-a9cc-47a5-99b8-e6e418e02b44


In [34]:
client.set.default_space(space_uid)

'SUCCESS'

In [38]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [39]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [40]:
model_details = client.repository.store_model(model='nutri-analyzer-model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='wqDBzE9bN4SnEL6XUswnZeQS_55exh8NK80putx-L51r',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'na-donotdelete-pr-rwpgsl31avjqx7'
object_key = 'Dataset.zip'

streaming_body_4 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7"})
model_id = client.repository.get_model_id(model_details)

SyntaxError: invalid syntax (645364500.py, line 3)

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='wqDBzE9bN4SnEL6XUswnZeQS_55exh8NK80putx-L51r',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'na-donotdelete-pr-rwpgsl31avjqx7'
object_key = 'Dataset.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='wqDBzE9bN4SnEL6XUswnZeQS_55exh8NK80putx-L51r',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'na-donotdelete-pr-rwpgsl31avjqx7'
object_key = 'Dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

In [ ]:
model_id

In [ ]:
client.repository.download(model_id,'nutrition.tar.gz')